In [1]:
import pandas as pd
import os
from itables import show
import numpy as np
import matplotlib.pyplot as plt

In [3]:
traffic = pd.read_csv('traffic.csv')
weather = pd.read_csv('weather_data.csv')
emissions = pd.read_csv('emission_huegelstr.csv')

In [5]:
weather

,Luftdruck[hPa],Relative Luftfeuchtigkeit[%],Temperatur[°C],Windgeschwindigkeit[m/s],Windrichtung[Grad],datetime
0,1004.0,61.0,11.5,2.6,235.0,2018-01-01 00:30:00
1,1004.0,62.7,11.0,2.7,244.0,2018-01-01 01:00:00
2,1004.0,64.0,10.7,2.0,229.0,2018-01-01 01:30:00
3,1004.0,65.5,10.4,2.1,235.0,2018-01-01 02:00:00
4,1005.0,65.0,10.2,2.6,243.0,2018-01-01 02:30:00
...,...,...,...,...,...,...
100143,1007.0,85.2,18.9,0.8,166.0,2023-09-18 08:00:00
100144,1007.0,81.6,19.4,0.9,157.0,2023-09-18 08:30:00
100145,1007.0,79.8,19.8,1.2,173.0,2023-09-18 09:00:00
100146,1007.0,76.4,20.4,1.2,176.0,2023-09-18 09:30:00


In [13]:
pred_data = traffic.merge(weather, how='left', left_on= 'roundedTimestamp', right_on='datetime').merge(emissions[['PM10','datetime']], how='left', left_on= 'roundedTimestamp', right_on='datetime' )
pred_data.drop(columns=['datetime_y', 'roundedTimestamp'], inplace = True)
show(pred_data)